In [1]:
library(ez)
library(car)
library(nlme)
require(multcomp)

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 

Loading required package: carData

Loading required package: multcomp

Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




# Correction post publication:
While the reported results in the paper are based on original file, we later found that the file which our results are based on was not up to date. Thus, here, we also provide the correct area sizes. The main difference to the paper is that now all the two-way interaction effects are significant. All other comparisons stayed the same.

In [2]:
df = read.csv("./out/Anova_Areas_total_v2.csv")

# In case you which to calculate the origianl resulst, use the follwoing file:
#df = read.csv("./data/Anova_Areas_total-as-reported-in-paper.csv")

In [3]:
# We are interested in the grip area which are caused by the fingers on the back. 
# Thus, we exclude the thumb in our ANOVAs. 
df = df[df$Finger != "Thumb",]

In [4]:
summary(df)

      Cond        Finger         PId        Phone      TouchedArea    
 seated :256   Index :128   Min.   : 3.00   n6 :128   Min.   :   8.0  
 walking:256   Little:128   1st Qu.: 6.75   opo:128   1st Qu.: 343.8  
               Middle:128   Median :11.50   s3 :128   Median : 507.5  
               Ring  :128   Mean   :11.62   s4 :128   Mean   : 622.7  
               Thumb :  0   3rd Qu.:16.25             3rd Qu.: 769.8  
                            Max.   :21.00             Max.   :3430.0  

In [5]:
df$Finger <- factor(df$Finger)
df$Cond <- factor(df$Cond)
df$Phone <- factor(df$Phone)
df$PId <- factor(df$PId)

# ANOVA on all Areas

In [6]:
aov<-ezANOVA(data = df,
       dv = TouchedArea,
       within = c(Finger, Phone, Cond),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)
aov 

$ANOVA
             Effect DFn DFd          F            p p<.05         ges
2            Finger   3  45 22.3907776 5.072824e-09     * 0.083174499
3             Phone   3  45 21.1481881 1.073480e-08     * 0.295495493
4              Cond   1  15  6.7070292 2.051138e-02     * 0.043744787
5      Finger:Phone   9 135  9.8012439 1.858350e-11     * 0.023168423
6       Finger:Cond   3  45 13.2805638 2.427770e-06     * 0.008249642
7        Phone:Cond   3  45  4.2482846 1.001009e-02     * 0.070630165
8 Finger:Phone:Cond   9 135  0.8760589 5.483866e-01       0.001866107

$`Mauchly's Test for Sphericity`
             Effect            W            p p<.05
2            Finger 2.866449e-01 4.409249e-03     *
3             Phone 2.472517e-01 1.875975e-03     *
5      Finger:Phone 5.951455e-04 2.249464e-04     *
6       Finger:Cond 4.414216e-01 4.806434e-02     *
7        Phone:Cond 1.251066e-01 3.192876e-05     *
8 Finger:Phone:Cond 2.773062e-05 6.016480e-09     *

$`Sphericity Corrections`
             Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2            Finger 0.6294120 2.058431e-06         * 0.7172129 4.929641e-07
3             Phone 0.5322061 1.517172e-05         * 0.5855710 6.588698e-06
5      Finger:Phone 0.4352247 4.431543e-06         * 0.6082638 9.687408e-08
6       Finger:Cond 0.6904253 5.788697e-05         * 0.8028741 1.820994e-05
7        Phone:Cond 0.4689027 3.996015e-02         * 0.5028787 3.653149e-02
8 Finger:Phone:Cond 0.3652047 4.679992e-01           0.4798922 4.898123e-01
  p[HF]<.05
2         *
3         *
5         *
6         *
7         *
8

In [7]:
aov$`Sphericity Corrections`$GGe * c(3, 3, 9, 3, 3, 9)
aov$`Sphericity Corrections`$GGe * c(45, 45, 135, 45, 45, 135)

[1] 1.888236 1.596618 3.917023 2.071276 1.406708 3.286842

[1] 28.32354 23.94928 58.75534 31.06914 21.10062 49.30263

In [8]:
lme_velocity <- lme(TouchedArea ~ Finger*Phone*Cond, data = df, random=~1|PId)

In [9]:
summary(glht(lme_velocity, linfct=mcp(Cond = "Tukey")), test = adjusted(type = "bonferroni"))

Warning message in mcp2matrix(model, linfct = linfct):
“covariate interactions found -- default contrast might be inappropriate”



	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = TouchedArea ~ Finger * Phone * Cond, data = df, 
    random = ~1 | PId)

Linear Hypotheses:
                      Estimate Std. Error z value Pr(>|z|)  
walking - seated == 0    237.8      116.4   2.042   0.0412 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


In [10]:
summary(glht(lme_velocity, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))

Warning message in mcp2matrix(model, linfct = linfct):
“covariate interactions found -- default contrast might be inappropriate”



	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = TouchedArea ~ Finger * Phone * Cond, data = df, 
    random = ~1 | PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
opo - n6 == 0   -161.1      116.4  -1.383 0.999566    
s3 - n6 == 0    -626.6      116.4  -5.381 4.44e-07 ***
s4 - n6 == 0    -453.9      116.4  -3.898 0.000581 ***
s3 - opo == 0   -465.5      116.4  -3.998 0.000384 ***
s4 - opo == 0   -292.9      116.4  -2.515 0.071367 .  
s4 - s3 == 0     172.6      116.4   1.483 0.829195    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


# ANOVA only on walking 

In [11]:
df2 = df[df$Cond == "walking",]
aov2<-ezANOVA(data = df2,
       dv = TouchedArea,
       within = c(Finger, Phone),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)
aov2

$ANOVA
        Effect DFn DFd         F            p p<.05        ges
2       Finger   3  45 23.995391 1.995993e-09     * 0.07088889
3        Phone   3  45  9.961246 3.741787e-05     * 0.29206024
4 Finger:Phone   9 135  5.902458 6.239983e-07     * 0.01913270

$`Mauchly's Test for Sphericity`
        Effect            W            p p<.05
2       Finger 2.302045e-01 1.219901e-03     *
3        Phone 6.570993e-02 5.793594e-07     *
4 Finger:Phone 2.874298e-05 6.773344e-09     *

$`Sphericity Corrections`
        Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2       Finger 0.5959310 2.018424e-06         * 0.6712148 5.523534e-07
3        Phone 0.4524288 2.550995e-03         * 0.4817366 2.027442e-03
4 Finger:Phone 0.3100803 2.315681e-03         * 0.3881527 8.906435e-04
  p[HF]<.05
2         *
3         *
4         *

In [12]:
aov2$`Sphericity Corrections`$GGe * c(3, 3, 9)
aov2$`Sphericity Corrections`$GGe * c(45, 45, 135)

[1] 1.787793 1.357286 2.790723

[1] 26.81689 20.35930 41.86085

In [13]:
lme2 <- lme(TouchedArea ~ Phone, data = df2, random=~1|PId)

In [14]:
summary(glht(lme2, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = TouchedArea ~ Phone, data = df2, random = ~1 | 
    PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
opo - n6 == 0  -608.56      72.61  -8.382  < 2e-16 ***
s3 - n6 == 0   -555.89      72.61  -7.656 1.15e-13 ***
s4 - n6 == 0   -732.83      72.61 -10.093  < 2e-16 ***
s3 - opo == 0    52.67      72.61   0.725   1.0000    
s4 - opo == 0  -124.27      72.61  -1.712   0.5219    
s4 - s3 == 0   -176.94      72.61  -2.437   0.0889 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


# ANOVA only on seated 

In [15]:
df2 = df[df$Cond == "seated",]

In [16]:
aov2<-ezANOVA(data = df2,
       dv = TouchedArea,
       within = c(Finger, Phone),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)

In [17]:
aov2

$ANOVA
        Effect DFn DFd         F            p p<.05        ges
2       Finger   3  45 17.938312 8.414487e-08     * 0.15409542
3        Phone   3  45 32.962218 1.996203e-11     * 0.44395642
4 Finger:Phone   9 135  5.242881 4.085076e-06     * 0.04563638

$`Mauchly's Test for Sphericity`
        Effect            W           p p<.05
2       Finger 0.4285755409 0.040908089     *
3        Phone 0.5070061858 0.097948410      
4 Finger:Phone 0.0005838472 0.000209635     *

$`Sphericity Corrections`
        Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2       Finger 0.7014106 4.692791e-06         * 0.8185556 9.644037e-07
3        Phone 0.7186104 8.685809e-09         * 0.8432697 5.859769e-10
4 Finger:Phone 0.4431759 1.104669e-03         * 0.6237701 1.754225e-04
  p[HF]<.05
2         *
3         *
4         *

In [18]:
aov2$`Sphericity Corrections`$GGe * c(3, 3, 9)
aov2$`Sphericity Corrections`$GGe * c(45, 45, 135)

[1] 2.104232 2.155831 3.988583

[1] 31.56348 32.33747 59.82875

In [19]:
lme2 <- lme(TouchedArea ~ Phone, data = df2, random=~1|PId)

In [20]:
summary(glht(lme2, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = TouchedArea ~ Phone, data = df2, random = ~1 | 
    PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
opo - n6 == 0  -189.86      39.53  -4.803 9.40e-06 ***
s3 - n6 == 0   -547.39      39.53 -13.847  < 2e-16 ***
s4 - n6 == 0   -377.67      39.53  -9.553  < 2e-16 ***
s3 - opo == 0  -357.53      39.53  -9.044  < 2e-16 ***
s4 - opo == 0  -187.81      39.53  -4.751 1.22e-05 ***
s4 - s3 == 0    169.72      39.53   4.293 0.000106 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)
